In [ ]:
import pandas as pd

Load Cleaned Data

In [31]:
# Load cleaned dataset
df_clean = pd.read_parquet('../data/processed/ames_clean.parquet', engine='pyarrow')

# Check the first few rows
print(df_clean.head())

   order        pid  ms_subclass ms_zoning  lot_frontage  lot_area street  \
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   
4      5  527105010           60        RL          74.0     13830   Pave   
5      6  527105030           60        RL          78.0      9978   Pave   
6      7  527127150          120        RL          41.0      4920   Pave   

  alley lot_shape land_contour  ... pool_qc  fence misc_feature misc_val  \
1  None       Reg          Lvl  ...    None  MnPrv         None        0   
2  None       IR1          Lvl  ...    None   None         Gar2    12500   
4  None       IR1          Lvl  ...    None  MnPrv         None        0   
5  None       IR1          Lvl  ...    None   None         None        0   
6  None       Reg          Lvl  ...    None   None         None        0   

  mo_sold yr_sold sale_type sale_condition  saleprice  log_saleprice  
1       6

In [32]:
# Check for missing values after initial cleaning
missing_values = df_clean.isnull().sum()
missing_values = missing_values[missing_values > 0]
print(f"Missing values in columns:\n{missing_values}")

Missing values in columns:
Series([], dtype: int64)


In [33]:
df_clean.dtypes

order                int64
pid                  int64
ms_subclass          int64
ms_zoning         category
lot_frontage       float64
                    ...   
yr_sold              int64
sale_type         category
sale_condition    category
saleprice            int64
log_saleprice      float64
Length: 83, dtype: object

In [34]:
df_clean['log_saleprice'].describe()

count    2483.000000
mean       11.980872
std         0.359110
min         9.456419
25%        11.766800
50%        11.957618
75%        12.206078
max        13.229570
Name: log_saleprice, dtype: float64

Drop Irrelavent/Leaky Columns

In [35]:
df_clean.columns

Index(['order', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
      

In [36]:
# one_hot encoding
df_clean = pd.get_dummies(df_clean, columns=['neighborhood'], drop_first=True)

In [37]:
# convert pool data to binary
df_clean['has_pool'] = (df_clean['pool_area'] > 0).astype(int)

In [38]:
# one_hot encoding
df_clean['all_conditions'] = df_clean[['condition_1', 'condition_2']].apply(
    lambda x: '_'.join(sorted(set(x))), axis=1
)

df_clean['all_exteriors'] = df_clean[['exterior_1st', 'exterior_2nd']].apply(
    lambda x: '_'.join(sorted(set(x))), axis=1
)

df_clean = pd.get_dummies(df_clean, columns=['all_conditions', 'all_exteriors'], drop_first=True)

In [39]:
cols_to_drop = [
    'order', 'pid', 'misc_val', 'misc_feature', 'alley', '3ssn_porch',
    'pool_qc', 'fence', 'utilities', 'mo_sold', 'yr_sold', 'garage_yr_blt',
    'condition_1', 'condition_2', 'exterior_1st', 'exterior_2nd',
    'pool_area',
]

y = df_clean['saleprice']
df_clean = df_clean.drop(columns=cols_to_drop + ['saleprice'])

In [41]:
df_clean.columns

Index(['ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area', 'street',
       'lot_shape', 'land_contour', 'lot_config', 'land_slope', 'bldg_type',
       ...
       'all_exteriors_Stucco_Wd Sdng', 'all_exteriors_Stucco_Wd Shng',
       'all_exteriors_Stucco_WdShing', 'all_exteriors_VinylSd',
       'all_exteriors_VinylSd_Wd Sdng', 'all_exteriors_VinylSd_Wd Shng',
       'all_exteriors_Wd Sdng', 'all_exteriors_Wd Sdng_Wd Shng',
       'all_exteriors_Wd Sdng_WdShing', 'all_exteriors_Wd Shng_WdShing'],
      dtype='object', length=174)

In [50]:
df_clean.to_parquet('../data/processed/ames_processed.parquet', index=False)